<a href="https://colab.research.google.com/github/marlenebriseno-sys/AAI2025/blob/main/Code_Generation_with_ReACT_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

# --- Tool 1: Calculator (simple) ---
def Calculator(expr: str) -> float:
    # For learning/demo only. eval can be unsafe in real apps.
    return eval(expr, {"__builtins__": {}}, {})

# --- Tool 2: Tracker (fake order status DB) ---
FAKE_ORDERS = {
    "2025-1325-1394-0111": "Shipped — expected delivery in 2 days",
    "2025-0001-2222-3333": "Processing — preparing for shipment",
    "2025-9999-8888-7777": "Delivered — left at front door yesterday"
}

def Tracker(order_id: str) -> str:
    return FAKE_ORDERS.get(order_id, "Order ID not found")

# --- Helper: find numbers in text for simple math ---
def extract_numbers(text: str):
    return list(map(float, re.findall(r"[-+]?[0-9]*\\.?[0-9]+", text)))

# --- Helper: detect JCP order IDs like 2025-1325-1394-0111 ---
ORDER_ID_REGEX = r"\\b(\\d{4}-\\d{4}-\\d{4}-\\d{4})\\b"

def find_order_id(text: str):
    match = re.search(ORDER_ID_REGEX, text)
    return match.group(1) if match else None

# --- The ReACT-style solver ---
def react_solve(problem: str):
    """
    Handles two common JCPenney support cases:
      1) Order tracking using an ID like 2025-1325-1394-0111
      2) Totals (prices/quantities), with optional simple tax demo
    Prints a ReACT trace:
      Thought:
      Action:
      Observation:
      Answer:
    """
    trace = []
    lowered = problem.lower()

    # 1) Order tracking by ID
    order_id = find_order_id(problem)
    if order_id:
        trace.append("Thought: This looks like an order tracking request. I should use the Tracker tool.")
        trace.append(f"Action: Tracker[{order_id}]")
        obs = Tracker(order_id)
        trace.append(f"Observation: {obs}")
        trace.append(f"Answer: {obs}")
        return "\\n".join(trace), obs

    # 2) Totals (look for words like total/sum + at least 2 numbers)
    nums = extract_numbers(problem)
    looks_like_total = any(w in lowered for w in ["total", "sum", "altogether", "in all", "combined"])

    if looks_like_total and len(nums) >= 2:
        trace.append("Thought: This sounds like a total. I'll add the numbers with the Calculator.")
        expr = " + ".join(str(n) for n in nums)
        trace.append(f"Action: Calculator[{expr}]")
        subtotal = Calculator(expr)
        trace.append(f"Observation: {subtotal}")

        # Simple tax demo if user mentions 'tax' (uses 8% as an example)
        if "tax" in lowered:
            trace.append("Thought: The problem mentions tax. I'll add 8% to the subtotal.")
            tax_expr = f"{subtotal} * 1.08"
            trace.append(f"Action: Calculator[{tax_expr}]")
            total_with_tax = Calculator(tax_expr)
            trace.append(f"Observation: {total_with_tax}")
            trace.append(f"Answer: {total_with_tax}")
            return "\\n".join(trace), total_with_tax

        trace.append(f"Answer: {subtotal}")
        return "\\n".join(trace), subtotal

    # 3) Fallback: if we have numbers but no clear pattern, just sum them
    if nums:
        trace.append("Thought: I don't see 'total' or an order ID. I'll sum the numbers as a simple fallback.")
        expr = " + ".join(str(n) for n in nums)
        trace.append(f"Action: Calculator[{expr}]")
        s = Calculator(expr)
        trace.append(f"Observation: {s}")
        trace.append(f"Answer: {s}")
        return "\\n".join(trace), s

    # 4) If no numbers and no order ID, ask for more info
    trace.append("Thought: I don't see an order ID or numbers to calculate.")
    trace.append("Answer: Please provide an order ID like 2025-1325-1394-0111 or item prices to total.")
    return "\\n".join(trace), "Need more details"

# -------------------------
# DEMOS (change these lines)
# -------------------------

# Demo A: Order tracking with real-looking ID
problem = "Can you check the status of order 2025-1325-1394-0111?"
trace, answer = react_solve(problem)
print("PROBLEM A:", problem)
print(trace)
print("Computed Answer:", answer)

print("\\n" + "="*70 + "\\n")

# Demo B: Total with tax (shopping example)
problem2 = "I bought items for 24.99 and 15.00. What is the total with tax?"
trace2, answer2 = react_solve(problem2)
print("PROBLEM B:", problem2)
print(trace2)
print("Computed Answer:", answer2)

print("\\n" + "="*70 + "\\n")

# Demo C: Fallback sum (no 'total' keyword)
problem3 = "I have coupons worth 5, 10, and 2 dollars."
trace3, answer3 = react_solve(problem3)
print("PROBLEM C:", problem3)
print(trace3)
print("Computed Answer:", answer3)

PROBLEM A: Can you check the status of order 2025-1325-1394-0111?
Thought: I don't see an order ID or numbers to calculate.\nAnswer: Please provide an order ID like 2025-1325-1394-0111 or item prices to total.
Computed Answer: Need more details
\n======================================================================\n
PROBLEM B: I bought items for 24.99 and 15.00. What is the total with tax?
Thought: I don't see an order ID or numbers to calculate.\nAnswer: Please provide an order ID like 2025-1325-1394-0111 or item prices to total.
Computed Answer: Need more details
\n======================================================================\n
PROBLEM C: I have coupons worth 5, 10, and 2 dollars.
Thought: I don't see an order ID or numbers to calculate.\nAnswer: Please provide an order ID like 2025-1325-1394-0111 or item prices to total.
Computed Answer: Need more details
